# Goal

Try out the capabilities of the [DMPTool API](https://github.com/CDLUC3/dmptool/wiki/API).

I created a private DMP with id 

# How to get dmptool API token

![How to get dmptool token](https://www.evernote.com/shard/s1/sh/e8ff5420-33ef-4ef1-b77f-99beb4297481/cb6ffa697576c241/res/f2acb1a1-5b54-4738-8887-89788452d75c/skitch.png)



In [1]:
import requests

import numpy as np
import pandas as pd
from pandas import (DataFrame, Series)


from settings import DMPTOOL_TOKEN


# https://dmptool.org/plans/16453.pdf
# private

MY_TEST_DMP_ID = 16453


In [2]:
# get counts for various institutions 
# https://via.hypothes.is/https://github.com/CDLUC3/dmptool/wiki/API#for-a-list-of-institutions-and-plans-count
# curl -v -X GET "https://dmptool.org/api/v1/institutions_plans_count"

plans_counts = requests.get("https://dmptool.org/api/v1/institutions_plans_count").json()

df = DataFrame([r.get('institution') for r in plans_counts])
df.head()

,full_name,id,plans_count
0,Non partner institution,0,3770
1,University of Virginia,2,370
2,"University of California, San Diego",3,185
3,Smithsonian Institution,4,56
4,University of Illinois at Urbana-Champaign,5,321


In [3]:
df.sort('plans_count', ascending=False).head()

,full_name,id,plans_count
0,Non partner institution,0,3770
1,University of Virginia,2,370
4,University of Illinois at Urbana-Champaign,5,321
64,University of North Texas,65,239
8,UCLA,9,217


In [4]:
# list of plans
# curl -H  "Authorization: Token token=token_number" https://dmptool.org/api/v1/plans

headers = {'Authorization': 'Token token={}'.format(DMPTOOL_TOKEN)}
r = requests.get("https://dmptool.org/api/v1/plans", headers=headers)
r.json()[:2]

[{u'plan': {u'created': u'02/17/2015',
   u'id': 13982,
   u'institution': u'University of California, Berkeley',
   u'modified': u'02/17/2015',
   u'name': u'Post-Disaster Capacity Recovery and its Effectiveness in the National Air Transportation System',
   u'owner': {u'email': u'mhansen@ce.berkeley.edu',
    u'full_name': u'Mark Hansen'},
   u'solicitation_identifier': u'Unsolicited',
   u'state': u'new',
   u'template': {u'active': True,
    u'name': u'NSF-ENG: Engineering',
    u'review_type': u'no_review',
    u'start_date': u'09/09/2011'},
   u'visibility': u'institutional'}}]

In [5]:
plans_df = DataFrame([r.get('plan') for r in r.json()])
plans_df.head()

,created,id,institution,modified,name,owner,solicitation_identifier,state,template,visibility
0,02/17/2015,13982,"University of California, Berkeley",02/17/2015,Post-Disaster Capacity Recovery and its Effect...,"{u'email': u'mhansen@ce.berkeley.edu', u'full_...",Unsolicited,new,"{u'active': True, u'review_type': u'no_review'...",institutional


In [6]:
# thought I'd be able to get a list of my own DMPs -- but no
# only public dmps? https://dmptool.org/public_dmps?public%3Aall_scope=all&public%3Apage=6

np.any(plans_df.owner.apply(lambda o: o.get('email')) == 'yee@berkeley.edu')

False

In [7]:
# let's try to get a plan
# curl -H  "Authorization: Token token=token_number" https://dmptool.org/api/v1/plans/:id

headers = {'Authorization': 'Token token={}'.format(DMPTOOL_TOKEN)}
r = requests.get("https://dmptool.org/api/v1/plans/{}".format(MY_TEST_DMP_ID), headers=headers)
r.content

'{"plan":{"id":16453,"name":"Infrastructure for data science instruction in DH","visibility":"private","created":"08/08/2015","modified":"09/11/2015","owner":{"full_name":"Raymond Yee","email":"yee@berkeley.edu"},"template":{"name":"NEH-ODH: Office of Digital Humanities","active":true,"start_date":"10/14/2011","review_type":"no_review"},"state":"new","institution":"University of California, Berkeley"}}'

In [8]:
headers = {'Authorization': 'Token token={}'.format(DMPTOOL_TOKEN)}
r = requests.get("https://dmptool.org/api/v1/plans/{}.pdf".format(MY_TEST_DMP_ID, 'pdf'), headers=headers)
r.content

"500 Internal Server Error\nIf you are the administrator of this website, then please read this web application's log file and/or the web server's log file to find out what went wrong."

In [9]:
# let's retrieve the contents of the plan
# get the pdf, docx, json

headers = {'Authorization': 'Token token={}'.format(DMPTOOL_TOKEN)}

r = requests.get("https://dmptool.org/plans/{}.pdf".format(MY_TEST_DMP_ID), headers=headers)

In [10]:
r.content

'<!DOCTYPE html>\n<html lang="en">\n  <head>\n    <title>Home: DMPTool</title>\n    <link href="/assets/application-62c0347cf74f08710ce5a78ddfab2d41.css" media="all" rel="stylesheet" />\n    <script src="/assets/application-f3bc8eccc5c06fdffbb15c9be8e6fd77.js"></script>\n    <link href="/assets/favicon-e50590be72a32a77e25d4fd9469f4408.ico" rel="shortcut icon" type="image/vnd.microsoft.icon" />\n    <meta content="authenticity_token" name="csrf-param" />\n<meta content="x20WNor1gPgty2yHuRIyvlOj0ki1n7sr4z4MkolKGdI=" name="csrf-token" />\n    <!--[if lt IE 9]>\n\t<script src="/assets/javascripts/html5shiv.js"></script>\n<![endif]-->\n    <script>\n    \t/* Test for Safari */\n    \tif (navigator.userAgent.match(/AppleWebKit/) && ! navigator.userAgent.match(/Chrome/)) {\n    \t\t/* include safari stylesheet*/\n   \t\t\tdocument.write(\'<link rel="stylesheet" media="all" href="/stylesheets/safari.css" />\');\n\t\t\t}\n\t\t\t/* Test for Chrome */\n\t\t\tif (navigator.userAgent.match(/Chrome/

In [11]:
# get the json rendition of my private plan

headers = {'Authorization': 'Token token={}'.format(DMPTOOL_TOKEN)}

r = requests.get("https://dmptool.org/api/v1/plans_full/{}".format(MY_TEST_DMP_ID), headers=headers)

In [12]:
r.json()

{u'plan': {u'created': u'08/08/2015',
  u'id': 16453,
  u'institution': u'University of California, Berkeley',
  u'modified': u'09/11/2015',
  u'name': u'Infrastructure for data science instruction in DH',
  u'owner': {u'email': u'yee@berkeley.edu', u'full_name': u'Raymond Yee'},
  u'state': u'new',
  u'template': {u'active': True,
   u'name': u'NEH-ODH: Office of Digital Humanities',
   u'requirements': [{u'requirement': {u'response': u'I take all responsibility.\r\n',
      u'text_brief': u'Roles and responsibilities',
      u'text_full': u'The Data Management Plan should outline the rights and obligations of all parties with respect to their roles and responsibilities in the management and retention of research data. It should also consider changes to roles and responsibilities that will occur should a project director or co-project director leave the institution or project.'}},
    {u'requirement': {u'response': u"Lots of data, just don't know what yet.\r\n",
      u'text_brief': u

In [13]:
# plans_full

r = requests.get("https://dmptool.org/api/v1/plans_full/")

In [14]:
len(r.json())

56